# Stateful Goroutines

To synchronize access to shared state across multiple goroutines, one option is to use the built-in synchronization features of goroutines and channels. This channel-based approach aligns with Go’s ideas of sharing memory by communicating and having each piece of data owned by exactly 1 goroutine.

Comments from [Go by Example](https://gobyexample.com).

In [1]:
:import "fmt"

In [2]:
:import "math/rand"

In [3]:
:import "sync/atomic"

In [4]:
:import "time"

## `structs` to Encapsulate Requests

In this example our state will be owned by a single goroutine. This will guarantee that the data is never corrupted with concurrent access. In order to read or write that state, other goroutines will send messages to the owning goroutine and receive corresponding replies. These readOp and writeOp structs encapsulate those requests and a way for the owning goroutine to respond.

In [5]:
type readOp struct {
    key  int
    resp chan int
}

In [6]:
type writeOp struct {
    key  int
    val  int
    resp chan bool
}

## Definitions

Count how many operations we perform.

In [7]:
var ops int64 = 0

The reads and writes channels will be used by other goroutines to issue read and write requests, respectively.

In [8]:
reads := make(chan *readOp)
writes := make(chan *writeOp)

(chan *main.writeOp)(0xc820018180)


## Goroutines

Here is the goroutine that owns the state, which is a map as in the previous example but now private to the stateful goroutine. This goroutine repeatedly selects on the reads and writes channels, responding to requests as they arrive. A response is executed by first performing the requested operation and then sending a value on the response channel resp to indicate success (and the desired value in the case of reads).

In [9]:
go func() {
    var state = make(map[int]int)
    for {
        select {
        case read := <-reads:
            read.resp <- state[read.key]
        case write := <-writes:
            state[write.key] = write.val
            write.resp <- true
        }
    }
}()

This starts 100 goroutines to issue reads to the state-owning goroutine via the reads channel. Each read requires constructing a readOp, sending it over the reads channel, and the receiving the result over the provided resp channel.

In [10]:
for r := 0; r < 100; r++ {
    go func() {
        for {
            read := &readOp{
                key:  rand.Intn(5),
                resp: make(chan int)}
            reads <- read
            <-read.resp
            atomic.AddInt64(&ops, 1)
        }
    }()
}

We start 10 writes as well, using a similar approach.

In [11]:
for w := 0; w < 10; w++ {
    go func() {
        for {
            write := &writeOp{
                key:  rand.Intn(5),
                val:  rand.Intn(100),
                resp: make(chan bool)}
            writes <- write
            <-write.resp
            atomic.AddInt64(&ops, 1)
        }
    }()
}
// let the goroutines work for a second
time.Sleep(time.Second)

## Collect the Ops Count

Let the goroutines work for a second. Finally, capture and report the ops count.

In [12]:
opsFinal := atomic.LoadInt64(&ops)

892297


Running our program shows that the goroutine-based state management example achieves about 800,000 operations per second.